In [ ]:
import pandas as pd

internal = pd.read_excel("C:/Users/user/Desktop/School 2024 Fall/Capstone/vodka/Vodka (Internal Sales).xlsx")
external = pd.read_excel("C:/Users/user/Desktop/School 2024 Fall/Capstone/vodka/Vodka (External Data).xlsx")
store_info = pd.read_excel("C:/Users/user/Desktop/School 2024 Fall/Capstone/vodka/Store Information.xlsx", skiprows=1)s

In [ ]:
store_info.head()

,State,Store Number,Store Name,Open Date,Price Zone/Market,Store Address,Store Size,Households (HH),% HH Income > $100K,Median HH Income,...,Cabernet Under $20,Cabernet $20-50,Cabernet Over $50,Chardonnay Under $20,Chardonnay Over $20,Wine - Sparkling,Pinot Noir Under $20,Pinot Noir Over $20,Sauvignon Blanc,French Champagne
0,DE,101,101\nDSW,1991-03-01,Delaware,"Northtowne Plaza\n691 Naamans Road\nClaymont, ...",Extra Large,61324,0.391152,78405,...,4,4,4,4,4,4,4,4,4,4
1,DE,102,102\nMSW,1991-07-01,Delaware,Milltown Shopping Center\n1325 McKennans Churc...,Small,61525,0.379764,78074,...,0,0,0,0,0,0,0,0,0,0
2,NJ,301,301\nCHW,1994-11-01,CHW/304,"The Plaza at Cherry Hill\n2100 Route 38, Suite...",Large,106097,0.408202,79937,...,3,4,2,4,2,4,4,3,4,3
3,MD,401,401\nBEW - Beltway,1996-11-01,BEW,"Beltway\n8727 Loch Raven Blvd\nTowson, MD 21286",Medium,120003,0.358491,72889,...,4,3,2,4,3,4,4,2,4,3
4,VA,202,202\nSpringfield,1997-11-01,N Virginia,Tower Shopping Center\n6801 Bland Street\nSpri...,Medium,126019,0.556646,109886,...,3,2,1,2,1,2,3,1,2,2


In [ ]:
store_info_clean = store_info.rename(columns={'State': 'Store State'})
external_clean = external.rename(columns={'item_code': 'Item Code', 'state': 'Store State', 'category': 'Category', 'sales_dollars_L52wk': 'external sales L52wk', 'points_of_distribution_L52wk': 'external points of distribution L52wk', 'package_type': 'Package Type', 'item_name': 'Item Name'})

In [ ]:
print(internal.columns)
print(external_clean.columns)
print(internal.shape[0])
print(external.shape[0])

Index(['Category', 'Store Number', 'Store State', 'Item Code', 'Item Name',
       'Package Type', 'Retail', 'L52W in Stock', 'Normalized Sales $ L52W',
       'Sales Bucket'],
      dtype='object')
Index(['Store State', 'Category', 'Item Code', 'Item Name', 'Package Type',
       'vintage', 'external sales L52wk',
       'external points of distribution L52wk'],
      dtype='object')
60973
22388


In [ ]:
# drop columns that only has one unique value
internal_cleaned = internal.loc[:, internal.nunique() > 1]
external_cleaned = external_clean.loc[:, external_clean.nunique() > 1]

print("Internal columns after dropping:", internal_cleaned.columns.tolist())
print("External columns after dropping:", external_cleaned.columns.tolist())

Internal columns after dropping: ['Store Number', 'Store State', 'Item Code', 'Item Name', 'Package Type', 'Retail', 'L52W in Stock', 'Normalized Sales $ L52W', 'Sales Bucket']
External columns after dropping: ['Store State', 'Item Code', 'Item Name', 'Package Type', 'external sales L52wk', 'external points of distribution L52wk']


In [ ]:
print(external_cleaned.isna().sum())

Store State                              0
Item Code                                0
Item Name                                0
Package Type                             0
external sales L52wk                     0
external points of distribution L52wk    0
dtype: int64


In [ ]:
# Perform a left join to retain all rows from internal_cleaned and bring in matching rows from external_cleaned
merged_data = pd.merge(
    internal_cleaned,
    external_cleaned,
    on=['Item Code', 'Store State', 'Package Type', 'Item Name'],
    how='left'
)
print(merged_data.shape[0])
print(merged_data.isna().sum())

60973
Store Number                                0
Store State                                 0
Item Code                                   0
Item Name                                   0
Package Type                                0
Retail                                      0
L52W in Stock                              15
Normalized Sales $ L52W                  6079
Sales Bucket                                0
external sales L52wk                     9416
external points of distribution L52wk    9416
dtype: int64


In [ ]:
# Identify rows in internal_cleaned without matching rows in external_cleaned
unmatched_rows = internal_cleaned[~internal_cleaned.set_index(['Item Code', 'Store State', 'Package Type', 'Item Name']).index.isin(
    external_cleaned.set_index(['Item Code', 'Store State', 'Package Type', 'Item Name']).index)]

print("Rows in internal_cleaned without a match in external_cleaned:")
print(unmatched_rows)
print("Number of unmatched rows:", unmatched_rows.shape[0])

Rows in internal_cleaned without a match in external_cleaned:
       Store Number Store State   Item Code                Item Name  \
0              2802          KS      503010             Barton Vodka   
1              2705          IN      503010             Barton Vodka   
2              2801          KS      503010             Barton Vodka   
11             2802          KS      503175             Barton Vodka   
29             1202          NV      503175             Barton Vodka   
...             ...         ...         ...                      ...   
60968          2603          MI  2126256533  Grit City Spirits Vodka   
60969          2601          MI  2126256533  Grit City Spirits Vodka   
60970          2605          MI  2126256533  Grit City Spirits Vodka   
60971          2602          MI  2126256533  Grit City Spirits Vodka   
60972          2301          CO  2168626750          Archangel Vodka   

      Package Type  Retail  L52W in Stock  Normalized Sales $ L52W  \
0  

In [ ]:
temp_merged = pd.merge(
    merged_data,
    store_info_clean,
    on=['Store Number', 'Store State'],
    how='outer'
)

# If necessary, filter out rows that were exclusively from store_info_clean
final_data = temp_merged[temp_merged['Item Code'].notna()]

# Display the count of NAs for comparison
print(final_data.isna().sum())

Store Number                                0
Store State                                 0
Item Code                                   0
Item Name                                   0
Package Type                                0
Retail                                      0
L52W in Stock                              15
Normalized Sales $ L52W                  6079
Sales Bucket                                0
external sales L52wk                     9416
external points of distribution L52wk    9416
Store Name                                  0
Open Date                                   0
Price Zone/Market                           0
Store Address                               0
Store Size                                  0
Households (HH)                             0
% HH Income > $100K                         0
Median HH Income                            0
Average Net Worth                           0
% Population w/ Bachelor's Degree +         0
% Hispanic                        

In [ ]:
# Identify stores in store_info_clean that are not in internal
stores_not_in_internal = store_info_clean[~store_info_clean['Store Number'].isin(internal['Store Number'])]

# Further filter to find stores with NA values in the 'Vodka' column
stores_with_na_vodka = stores_not_in_internal[stores_not_in_internal['Vodka'].isna()]

# Display the result
print("Stores in store_info_clean but not in internal, with NA for Vodka:")
print(stores_with_na_vodka[['Store Number', 'Store State', 'Store Name']])
print("Total stores not in internal with NA for Vodka:", stores_with_na_vodka.shape[0])

Stores in store_info_clean but not in internal, with NA for Vodka:
     Store Number Store State                Store Name
4             202          VA          202\nSpringfield
6             205          VA               205\nMcLean
7             203          VA           203\nAlexandria
8             201          VA            201\nChantilly
10            204          VA             204\nSterling
11            206          VA          206\nS. Richmond
12            207          VA          207\nW. Richmond
13            208          VA       208\nFredericksburg
14            210          VA             210\nVA Beach
15            211          VA           211\nChesapeake
17            602          NC         602\nBattleground
18            603          NC           603\nGreensboro
19            604          NC        604\nWinston Salem
20            605          NC   605\nPark Towne Village
22            606          NC                 606\nCary
23            609          NC        

In [ ]:
# Create sets of unique (Store State, Item Code) pairs in both DataFrames
internal_pairs = set(internal_cleaned[['Store State', 'Item Code']].drop_duplicates().apply(tuple, axis=1))
external_pairs = set(external_cleaned[['Store State', 'Item Code']].drop_duplicates().apply(tuple, axis=1))

# Find pairs in internal but not in external
unique_to_internal = internal_pairs - external_pairs

# Count the number of unique pairs in internal but not in external
count_unique_to_internal = len(unique_to_internal)

# Display the result
print("Number of unique (Store State, Item Code) pairs in internal_cleaned but not in external_cleaned:", count_unique_to_internal)

Number of unique (Store State, Item Code) pairs in internal_cleaned but not in external_cleaned: 3393


In [ ]:
sales_included = final_data[final_data['Sales Bucket'] == 'Sales included']
sales_excluded = final_data[final_data['Sales Bucket'] == 'Sales excluded for model test']

In [ ]:
print(sales_included.shape[0])
print(sales_excluded.shape[0])
print(final_data['Sales Bucket'].value_counts())

54894
6079
Sales included                   54894
Sales excluded for model test     6079
Name: Sales Bucket, dtype: int64


In [ ]:
final_data.to_csv('final_data.csv', index=False)
sales_included.to_csv('sales_included.csv', index=False)
sales_excluded.to_csv('sales_excluded.csv', index=False)